In [ ]:
import numpy as np
import pandas as pd
import altair as alt
from spinorama.load_webplotdigitizer import parse_graph_freq_webplotdigitizer
from spinorama.graph import graph_freq, graph_spinorama, graph_params_default
from spinorama.load_misc import graph_melt
from spinorama.load import spin_compute_di_eir
from spinorama.compute_normalize import normalize_cea2034, normalize_mean

In [ ]:
speaker_name = "Perlisten s7t"
origin = "misc-audioholics"

data_dir = "./datas/measurements/{}/{}".format(speaker_name, origin)
high_file = "{}/high/wpd.json".format(data_dir)
low_file = "{}/low/wpd.json".format(data_dir)

title_low, df_low = parse_graph_freq_webplotdigitizer(low_file)
title_high, df_high = parse_graph_freq_webplotdigitizer(high_file)

In [ ]:
high_freq_transition  = df_high.loc[df_high.Measurements=='On Axis'].Freq.min()
high_db_transition    = df_high.loc[df_high.Measurements=='On Axis'].dB.values[0]
high_db_transition_lw = df_high.loc[df_high.Measurements=='Listening Window'].dB.values[0]
high_db_transition_er = df_high.loc[df_high.Measurements=='Early Reflections'].dB.values[0]
high_db_transition_sp = df_high.loc[df_high.Measurements=='Sound Power'].dB.values[0]


print('High: freq min {}'.format(high_freq_transition))
print('High: dB {}'.format(high_db_transition))

In [ ]:
low_freq_transition = df_low.loc[(df_low.Freq<high_freq_transition)].Freq.max()
low_db_transition = df_low.loc[(df_low.Freq<high_freq_transition)].dB.values[-1]

print('Low: freq min {}'.format(low_freq_transition))
print('Low: dB {}'.format(low_db_transition))

In [ ]:
def adapt(onaxis, low_db_transition, high_db_transition, target_db):
    max_pos = np.argmax(onaxis.dB.loc[onaxis.Freq<100])
    values = onaxis.dB.values
    delta = high_db_transition-low_db_transition
    result = [v+delta if i<=max_pos else v+delta+(i-max_pos)*(target_db-high_db_transition)/(len(values)-max_pos) for i, v in enumerate(values)]
    return result

df_low_match = df_low[(df_low.Freq<high_freq_transition)]
df_low_onaxis = pd.DataFrame({
    'Freq': df_low_match.Freq,
    'dB': df_low_match.dB-(low_db_transition-high_db_transition),
    'Measurements': df_low_match.Measurements
})
df_low_lw = pd.DataFrame({
    'Freq': df_low_match.Freq,
    'dB': adapt(df_low_match, low_db_transition, high_db_transition, high_db_transition_lw),
    'Measurements': 'Listening Window'
})
df_low_er = pd.DataFrame({
    'Freq': df_low_match.Freq,
    'dB': adapt(df_low_match, low_db_transition, high_db_transition, high_db_transition_er),
    'Measurements': 'Early Reflections'
})
df_low_sp = pd.DataFrame({
    'Freq': df_low_match.Freq,
    'dB': adapt(df_low_match, low_db_transition, high_db_transition, high_db_transition_sp),
    'Measurements': 'Sound Power'
})
df_low_final = pd.concat([df_low_onaxis, df_low_lw, df_low_sp, df_low_er], ignore_index=True).reset_index()
df = pd.concat([df_low_final, df_high], ignore_index=True).reset_index()
spin_uneven = df.pivot_table(index='Freq', columns='Measurements', values='dB', aggfunc=max).reset_index()
# spin_full = spin_compute_di_eir(speaker_name, 'title', spin_uneven)
mean = normalize_mean(spin_uneven)

In [ ]:
graph_params = graph_params_default.copy()
graph_params['ymax'] = 100
graph_params['ymin'] = 50

In [ ]:
graph_spinorama(df, graph_params)

In [ ]:
df['Phase'] = 0.0
for col_name, file_name in (
    ('On Axis', 'On Axis'),
    ('Listening Window', 'LW'),
    ('Sound Power', 'SP'),
    ('Early Reflections', 'ER'),
):
    df.loc[df.Measurements==col_name].to_csv(path_or_buf='{}/{}.txt'.format(data_dir, file_name), sep=' ', columns=['Freq', 'dB', 'Phase'],  index=False, header=False)